<a href="https://colab.research.google.com/github/ashiqur9/Facial_Recognition_Using_Keras/blob/master/Create_face_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I use bing search api to create face dataset. This gonna take few time to create dataset

In [0]:
 
# import the necessary packages
from requests import exceptions
import argparse
import requests
import cv2
import os

In [0]:
query="Scarlett Johansson"
output="/content/drive/My Drive/face/Scarlett_Johansson"
os.mkdir(output)

In [0]:
 
API_KEY = "API_KEY"
MAX_RESULTS = 1000
GROUP_SIZE = 100

In [0]:
 
# set the endpoint API URL
URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"

In [0]:
 
# when attempting to download images from the web both the Python
# programming language and the requests library have a number of
# exceptions that can be thrown so let's build a list of them now
# so we can filter on them
EXCEPTIONS = set([IOError, FileNotFoundError,
    exceptions.RequestException, exceptions.HTTPError,
    exceptions.ConnectionError, exceptions.Timeout])

In [0]:
 
#How to (quickly) build a deep learning image dataset
# store the search term in a convenience variable then set the
# headers and search parameters
term = query
headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE}
# make the search
print("[INFO] searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()
# grab the results from the search, including the total number of
# estimated results returned by the Bing API
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("[INFO] {} total results for '{}'".format(estNumResults,
    term))
# initialize the total number of images downloaded thus far
total = 0

[INFO] searching Bing API for 'Scarlett Johansson'
[INFO] 852 total results for 'Scarlett Johansson'


In [0]:
 
# loop over the estimated number of results in `GROUP_SIZE` groups
for offset in range(0, estNumResults, GROUP_SIZE):
    # update the search parameters using the current offset, then
    # make the request to fetch the results
    print("[INFO] making request for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    print("[INFO] saving images for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    # loop over the results
    for v in results["value"]:
        # try to download the image
        try:
            # make a request to download the image
            print("[INFO] fetching: {}".format(v["contentUrl"]))
            r = requests.get(v["contentUrl"], timeout=30)
            # build the path to the output image
            ext = v["contentUrl"][v["contentUrl"].rfind("."):]
            p = os.path.sep.join([output, "{}{}".format(
                str(total).zfill(8), ext)])
            # write the image to disk
            f = open(p, "wb")
            f.write(r.content)
            f.close()
        # catch any errors that would not unable us to download the
        # image
        except Exception as e:
            # check to see if our exception is in our list of
            # exceptions to check for
            if type(e) in EXCEPTIONS:
                print("[INFO] skipping: {}".format(v["contentUrl"]))
                continue
# try to load the image from disk
        image = cv2.imread(p)
        # if the image is `None` then we could not properly load the
        # image from disk (so it should be ignored)
        if image is None:
            print("[INFO] deleting: {}".format(p))
            os.remove(p)
            continue
        # update the counter
        total += 1

[INFO] making request for group 0-100 of 852...
[INFO] saving images for group 0-100 of 852...
[INFO] fetching: https://media1.popsugar-assets.com/files/thumbor/dhARACfJ2YB_DcGNKFgESJQJCVM/fit-in/1024x1024/filters:format_auto-!!-:strip_icc-!!-/2018/07/30/682/n/1922153/1a78366e12b7a423_GettyImages-955779892/i/Scarlett-Johansson.jpg
[INFO] fetching: https://www.hawtcelebs.com/wp-content/uploads/2018/11/scarlett-johansson-at-people-s-choice-awards-2018-in-santa-monica-11-11-2018-6.jpg
[INFO] fetching: https://tomandlorenzo.com/wp-content/uploads/2018/11/Scarlett-Johansson-2018-People-Choice-Awards-Red-Carpet-Fashion-Versace-Tom-Lorenzo-Site-6.jpg
[INFO] fetching: https://i.dailymail.co.uk/i/newpix/2018/04/25/15/4B82698D00000578-5656197-image-a-37_1524665856145.jpg
[INFO] fetching: https://www.hawtcelebs.com/wp-content/uploads/2018/04/scarlett-johansson-at-avengers-infinity-war-premiere-in-los-angeles-04-23-2018-4.jpg
[INFO] fetching: https://celebmafia.com/wp-content/uploads/2018/11/scarl